In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import yfinance as yf

## Equity Class

Class Attributes include:
- asset
- df

Class Methods include:
- historical_volatility (The calculation was taken from [here](https://www.investopedia.com/ask/answers/021015/how-can-you-calculate-volatility-excel.asp))
- min_max_scale

Note:
- yfinance module can be found [here](https://libraries.io/pypi/fix-yahoo-finance)

In [2]:
class equity():
    
    #Defining Underlying Asset
    #Name: Pass the desired ticker as string all uppercase e.g. 'GOOG'
    #Period: Set the desired study period, e.g. '30d', '2y'.
        #Note: Use 'max' period to get the entire available historical dataset
    
    def __init__(self, name, period):
        self.asset = yf.Ticker(name)
        self.df = self.asset.history(period)
                
    def historical_volatility(self, lookback_in_days):
        def _return_on_price(current_price, previous_price):
            return (current_price/previous_price) - 1
        self.df['Close Shift'] = self.df['Close'].shift(1)
        self.df['Close Price Return'] = self.df.apply(lambda row : _return_on_price(row['Close'], row['Close Shift']), axis = 1)
        self.df['Rolling Volatility'] = np.sqrt(lookback_in_days) * self.df['Close Price Return'].rolling(lookback_in_days).std() * 100
        return self
    
    def min_max_scale(self, field_name):
        min_max_scaler = preprocessing.MinMaxScaler()
        self.df[field_name + ' Scaled'] = min_max_scaler.fit_transform(self.df[[field_name]])
        return self